Esta es una demostración de cómo se va a realizar el proceso de ETL en el módulo ETL.py.
Por lo tanto, se va a trabajar únicamente con un dataframe a modo de muestra.


In [111]:
import pandas as pd

In [ ]:
df_amazon = pd.read_csv('/datasets/amazon_prime_titles.csv')

In [ ]:
df_amazon.columns

Se definen qué columnas se van a utilizar para las consultas pertinentes.

*show_id*: Número de ID para cada serie o película <br>
*type*: Define si es serie o película <br>
*title*: Título <br>
*cast*: Reparto de actores y actrices <br>
*release_year*: Fecha de lanzamiento <br>
*duration*: Duración de la misma (en minutos o en temporadas) <br>
*listed_in*: Categoría/s <br>
 
Ya que no son necesarias para ninguna consulta, se descartan las siguientes columnas:

*director*: Nombre del director<br>
*country*: País de realización<br>
*date_added*: Fecha en que fueron introducidos los datos al dataset<br>
*rating*: Clasificación de la película (ATP, +13, etc)<br>
*description*: Una breve sinopsis<br>

In [ ]:
#Se guardan las columnas a utilizar en una lista

columns = ['type', 'title', 'cast',
       'release_year', 'duration', 'listed_in']

In [ ]:
#Se guarda una copia de cada dataset

df_amazon_copy = df_amazon.copy()

In [ ]:
#Si las columnas no están en la lista columns, se dropean del dataframe.

df_amazon_copy.drop(columns=[col for col in df_amazon_copy if col not in columns], inplace=True)

In [ ]:
#Podemos ver que no es tan grande la cantida de nulos, superando apenas el 10% solo en la columna cast
# por lo que no es necesario dropear nulos ni llenarlos (de momento) cruzando datos con las demás plataformas

df_amazon_copy.isna().sum()

In [ ]:
#Comprobamos con el output de la tabla anterior si hay duplicados, vemos que no los hay

df_amazon_copy.drop_duplicates().info()

In [ ]:
df_amazon_copy['listed_in'].str.contains('Comedy').sum()

In [ ]:
#Vamos a separar duration en cantidad, y luego en la unidad en la que está medido(minutos, temporadas)

df_amazon_copy

In [32]:
#Observamos que se separa la unidad por un espacio en blanco
df_amazon_copy.duration

NameError: name 'df_amazon_copy' is not defined

In [ ]:
# El método str.split divide la columna en dos, separando el contenido, por defecto con un espacio, y con
# el parámetro expand=True, se expande en un dataframe con dos columnas.
# Luego el método join hace la unión del df original con las columnas.

df_amazon_copy = df_amazon_copy.join(df_amazon_copy.duration.str.split(expand=True))


In [ ]:
df_amazon_copy.shape

In [ ]:
# Dropeamos la vieja columna duration
# Renombramos las dos columnas nuevas, una con duration(cantidad de minutos o temporadas)
# y la otra con unit(unidad de tiempo, minutos o temporadas)

df_amazon_copy.drop(columns='duration', inplace=True)
df_amazon_copy.rename(columns={0:'duration',
                                1:'unit'}, inplace=True)
df_amazon_copy.sample(5)

In [ ]:
df_amazon_copy.shape

In [ ]:
#Agregamos a qué plataforma pertenece este dataframe

df_amazon_copy['platform'] = 'Amazon'

In [ ]:
# Aplicamos el método title para normalizar mayúsculas y minúsculas
# Salteamos duration y release_year porque son números y no se puede aplicar este método
for n in df_amazon_copy:
    if n=='duration' or n=='release_year':
        pass
    else:
       df_amazon_copy[f'{n}'] = df_amazon_copy[f'{n}'].str.title()

In [ ]:
# Asignamos el tipo de dato de duration 

df_amazon_copy.duration = df_amazon_copy.duration.astype('int64')

In [ ]:
df_amazon_copy.head()

Terminamos! Tenemos el df normalizado en todos sus datos. Y únicamente tenemos las columnas necesarias.